# Introduction

**This notebook reads a directory of pdf files, extracts the texts using an OCR package (pytesseract), finetunes an OpenAI Curie model, and runs the model on a subdirectory of pdf files. <br><br>The structure of the notebook goes like this:**
1. [Setup](#Setup)
2. [OCR Text Extraction](#OCR-Text-Extraction)
3. [Training Data Preparation](#Training-Data-Preparation)
4. [Fine Tuning OpenAI's Curie Model](#Fine-Tuning-OpenAI's-Curie-Model)
5. [Validation Data Preparation](#Validation-Data-Preparation)
6. [Running Fine Tuned Model](#Running-Fine-Tuned-Model)

# Setup

In [1]:
from pdf2image import convert_from_bytes
import pytesseract
import os
import pandas as pd
from tqdm import tqdm
import re
import openai
import tiktoken
import random
import json
import fitz
import time
import requests
from requests.packages.urllib3.util import ssl_
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [2]:
os.getcwd()

'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\03-CODE\\Base principal'

**Set own root directory.**

In [3]:
root = r'C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE'
os.chdir(root)

**We also define other important directories.**

In [4]:
data_raw = root + r'\01-DATA_PERU\01-DATA_RAW'
data_pro = root + r'\01-DATA_PERU\02-DATA_PROCESSED'
documentation = root + r'\01-DATA_PERU\04-DATA_DOCUMENTATION'

# PDF Document Scraping

In [5]:
OSCE_finetune_sample = pd.read_stata(data_pro + r'\OSCE_finetune_sample.dta', convert_dates=True, convert_categoricals=True, index_col=None, convert_missing=False, preserve_dtypes=True, columns=None, order_categoricals=True, chunksize=None, iterator=False, compression='infer', storage_options=None)
OSCE_finetune_sample

,codigoconvocatoria,n_cod_contrato,urlcontrato,fecha_suscripcion_contrato,year_suscripcion,n_item1,ruc_proveedor1,ruc_destinatario_pago1,n_item2,ruc_proveedor2,...,ruc_destinatario_pago9,n_item10,ruc_proveedor10,ruc_destinatario_pago10,n_item11,ruc_proveedor11,ruc_destinatario_pago11,n_item12,ruc_proveedor12,ruc_destinatario_pago12
0,590320,2012614,http://zonasegura.seace.gob.pe/documentos//srv...,2020-08-11,2020.0,168,20536390201,20536390201,NaN,,...,,NaN,,,NaN,,,NaN,,
1,590150,2016288,http://zonasegura.seace.gob.pe/documentos//srv...,2020-06-26,2020.0,9,20136961528,20136961528,NaN,,...,,NaN,,,NaN,,,NaN,,
2,590320,2020775,http://zonasegura.seace.gob.pe/documentos//srv...,2020-06-26,2020.0,30,20100287791,20100287791,214.0,20100287791,...,,NaN,,,NaN,,,NaN,,
3,590320,2016452,http://zonasegura.seace.gob.pe/documentos//srv...,2020-07-02,2020.0,117,20100018625,20100018625,NaN,,...,,NaN,,,NaN,,,NaN,,
4,590320,2005726,http://zonasegura.seace.gob.pe/documentos//srv...,2020-08-11,2020.0,353,20347268683,20347268683,NaN,,...,,NaN,,,NaN,,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,648722,2003441,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2020-09-16,2020.0,1,L0556212988,L0556212988,NaN,,...,,NaN,,,NaN,,,NaN,,
296,680736,2042325,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2021-02-05,2021.0,2,10296723610,10296723610,NaN,,...,,NaN,,,NaN,,,NaN,,
297,641838,2001425,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2020-09-16,2020.0,1,10763052848,10763052848,NaN,,...,,NaN,,,NaN,,,NaN,,
298,681470,2032866,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2020-12-29,2020.0,1,10738273660,10738273660,NaN,,...,,NaN,,,NaN,,,NaN,,


In [6]:
save_directory = documentation + r'\Finetune_sample\downloaded_pdfs'

# Create the directory if it doesn't exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [7]:
# Override SSL settings
# ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'

# Disable only DH cipher
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'HIGH:!DH:!aNULL'

failed_downloads_df = pd.DataFrame(columns=['n_cod_contrato', 'urlcontrato', 'failed_download'])

# Download and save PDFs
for index, row in tqdm(OSCE_finetune_sample.iterrows(), total=OSCE_finetune_sample.shape[0]):
    url = row['urlcontrato']
    contract_code = row['n_cod_contrato']
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=InsecureRequestWarning)
            response = requests.get(url, verify=False)
            if response.status_code == 200:
                # Generate a name for the PDF based on the contract's code
                filename = os.path.join(save_directory, f'pdf_{contract_code}.pdf')
                with open(filename, 'wb') as f:
                    f.write(response.content)
            else:
                failed_downloads_df = pd.concat([failed_downloads_df,pd.DataFrame({'n_cod_contrato': contract_code, 'urlcontrato': url, 'failed_download': 1}, index=[0])], ignore_index=True)
                #failed_downloads_df = failed_downloads_df.append({'Contract Code': contract_code, 'URL': url}, ignore_index=True)
                print(f'Failed to download PDF {contract_code} from {url}. Status code: {response.status_code}')
    except requests.exceptions.SSLError as e:
        print(f"An SSL error occurred: {e} in contract {contract_code} with url: {url}")

  1%|▌                                                                                 | 2/300 [00:00<00:23, 12.53it/s]

Failed to download PDF 2012614 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/0b59b1dc-7fc8-409f-ad1e-3e201c7c4d1b. Status code: 404
Failed to download PDF 2016288 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/2259b2b9-d55d-493b-a985-68daa1e2dc94. Status code: 404
Failed to download PDF 2020775 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/40bdce0e-f8a5-415e-817c-e562d515707a. Status code: 404


  2%|█▋                                                                                | 6/300 [00:00<00:23, 12.49it/s]

Failed to download PDF 2016452 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/45f31d4c-b6b0-47f4-9eb1-7f8e79e00b0c. Status code: 404
Failed to download PDF 2005726 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/48052223-bdfb-4d13-b3f2-d6aa555dd139. Status code: 404
Failed to download PDF 2013162 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/75617b27-265f-41f2-9b61-fddaa610c1b2. Status code: 404



  3%|██▏                                                                               | 8/300 [00:00<00:24, 12.10it/s]

...

  5%|███▊                                                                             | 14/300 [00:04<02:06,  2.25it/s]

Failed to download PDF 2003522 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/e2c881d2-5e66-450c-a74b-f9b5666186ec. Status code: 404
Failed to download PDF 2053289 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/e70a7122-b993-46d9-aa34-48879f2f341b. Status code: 404
Failed to download PDF 2008835 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/ef641fe0-926d-4925-bf71-9560a3b9405a. Status code: 404
Failed to download PDF 2006453 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/f9086de0-28ef-4a77-848d-5122f5710259. Status code: 404


 49%|██████████████████████████████████████▉                                         | 146/300 [32:20<51:52, 20.21s/it]

Failed to download PDF 1291759 from http://zonasegura.seace.gob.pe/documentos/mon\docs\contratos\2020\37\364370114082020112220.pdf. Status code: 404


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [45:35<00:00,  9.12s/it]


In [ ]:
failed_downloads_df.drop('urlcontrato',
  axis='columns', inplace=True)

# OCR Text Extraction

**OCR extraction with Tesseract in Windows requires to have the program installed from this [link](https://github.com/UB-Mannheim/tesseract/wiki).<br> After installing, we need to specify the location of the exe file as below.**

In [9]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

**We first define the directory where the pdfs are located.**

In [10]:
pdfs = documentation + r'\Finetune_sample\downloaded_pdfs'

**We span through the files of the base pdf directory, and its subdirectories, storing the filepath of each document.**

In [11]:
filenames = []

for base, dirs, files in os.walk(pdfs):
    for filename in files:
        if (filename.lower().endswith('.pdf')):
            filenames.append(os.path.join(base, filename))

random.choices(filenames, k=5)

['C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Finetune_sample\\downloaded_pdfs\\pdf_2036970.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Finetune_sample\\downloaded_pdfs\\pdf_2021803.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Finetune_sample\\downloaded_pdfs\\pdf_2034430.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Finetune_sample\\downloaded_pdfs\\pdf_2034152.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Finetune_sample\\downloaded_pdfs\\pdf_2004049.pdf']

**We open each path and extract the text using the fitz package (a simple pdf reader). <br>Then we identify if the pdf was a scanned document (an image without *"highlight-able"* text) by setting a length threshold. <br>Only if the read pdf's text has a length below the threshold, we proceed by re-extracting the text using the Tesseract OCR package. <br>Each text is stored in a dataframe containing its filename, text, extraction type and file id.**

In [26]:
# Create a dataframe to store the texts of each PDF
pdf_texts_df = pd.DataFrame(columns = ['filename', 'text', 'extraction_type'])
broken_pdfs_df = pd.DataFrame(columns=['n_cod_contrato', 'broken_pdf'])

# Loop through every file in the directory
for filename in tqdm(filenames):
    if filename.lower().endswith('.pdf'):
        try:
        
            # Read the text directly from the PDF file
            reader = fitz.open(filename)
            pdf_text = ''

            for page in reader:
                pdf_text+=page.get_text()+' '
            
            pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'PDF_Reader'
            
            if len(re.sub(r'[^a-zA-Z]', '', pdf_text))<1000:
            
                # Open the PDF file
                with open(filename, 'rb') as file:
                    pdf_bytes = file.read()

                # Convert the PDF to images
                images = convert_from_bytes(pdf_bytes)

                # Use OCR to extract text from each image/page
                pdf_text = ''
                for i, image in enumerate(images):
                    text = pytesseract.image_to_string(image)
                    pdf_text+=text+' '
            
                pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'OCR'

            # Clean the extracted text

            clean_text = re.sub('\$+', ' ', pdf_text)  # Replace multiple \$ with a space
            clean_text = re.sub('\n+', ' ', clean_text)  # Replace multiple newlines with one space
            clean_text = re.sub('\.+', '.', clean_text)  # Replace multiple . with one space
            clean_text = re.sub('\,+', ',', clean_text)  # Replace multiple newlines with one space
            clean_text = clean_text.replace(';', ' ')  # Replace semicolons with spaces
            clean_text = re.sub(' +', ' ', clean_text)  # Replace multiple spaces with one
            clean_text = re.sub(r'[^a-zA-ZÀ-ÿ0-9 \,\.\/\:]', '', clean_text)
        
            # Store the joined text in the dataframe, using the filename (without .pdf) as the key
            pdf_texts_df.loc[filenames.index(filename), 'filename'] = filename[:-4]
            pdf_texts_df.loc[filenames.index(filename), 'text'] = clean_text
            
        except Exception as e:
            error_message = str(e)
            if 'cannot open broken document' in error_message:  # Replace 'FileDataError' with the actual error message you expect
                print(f"A FileDataError occurred: {e} in {filename}")
                broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'n_cod_contrato': [filename.replace(pdfs+'\\pdf_', '').replace('.pdf','')], 'broken_pdf': 1}, index=[0])], ignore_index=True)
                #broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'Contract Code': filename.str.replace(pdfs+'\\pdf_', '', regex=False)}, index=[0])], ignore_index=True)
            else:
                print(f"An unspecified error occurred: {e} in {filename}")

 20%|████████████████▌                                                                | 58/284 [13:36<40:07, 10.65s/it]

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Finetune_sample\downloaded_pdfs\pdf_1243927.pdf


 55%|███████████████████████████████████████████▉                                    | 156/284 [29:42<22:35, 10.59s/it]

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Finetune_sample\downloaded_pdfs\pdf_2011511.pdf


100%|████████████████████████████████████████████████████████████████████████████████| 284/284 [54:37<00:00, 11.54s/it]


In [28]:
pdf_texts_df['n_cod_contrato'] = pdf_texts_df['filename'].str.replace(pdfs+'\\pdf_', '', regex=False)
pdf_texts_df.drop('filename', axis='columns', inplace=True)

In [37]:
training_dfs = data_pro + r'\training_dfs'

# Create the directory if it doesn't exist
if not os.path.exists(training_dfs):
    os.makedirs(training_dfs)

In [35]:
display(failed_downloads_df)
display(broken_pdfs_df)
display(pdf_texts_df)

,n_cod_contrato,failed_download
0,2012614,1
1,2016288,1
2,2020775,1
3,2016452,1
4,2005726,1
5,2013162,1
6,2012290,1
7,2038676,1
8,2052075,1
9,2012610,1


,n_cod_contrato,broken_pdf
0,1243927,1
1,2011511,1


,text,extraction_type,n_cod_contrato
0,"EMAPACOP .A, EMPRESA MUNICIPAL OF AGUA POTABL...",OCR,1203413
1,".. , . ..,.. ,.. . ,,. IAFAS del Ejército del ...",PDF_Reader,1204449
2,2 EMAPACOP S.A. 8 EMPRESA MUNICIPAL DE AGUA PO...,OCR,1206365
3,Sistema Integrado de Gestidn Administrativa Mo...,OCR,1206762
4,Subasta Inversa Electronica N 0122018CENARES/M...,OCR,1207214
...,...,...,...
279,Ministerio SENASA oe stead PROGRAMA DE DESAR...,OCR,2071439
280,PERU de Trabajo ae Sela o Tu ate awe : feet ...,OCR,2086854
281,isteri Seguro Social de Salud Rex EsSalud y Pr...,OCR,2098813
282,Sistema Integrado de Gestién Administrativa Mé...,OCR,2107267


In [38]:
pdf_texts_df.to_excel(training_dfs + r'\pdf_texts.xlsx', index = False)
failed_downloads_df.to_excel(training_dfs + r'\failed_downloads.xlsx', index = False)
broken_pdfs_df.to_excel(training_dfs + r'\broken_pdfs.xlsx', index = False)

**Inside our PDF directory, there is a subdirectory specific to the data validation PDFs. <br>We create a file id column to identify which files are from the validation data directory.**

In [8]:
pdf_texts_df_val = pdf_texts_df[pdf_texts_df['file_id'].str.startswith('Validation PDFs')].reset_index(drop=True)
pdf_texts_df_val['filename'] = pdf_texts_df_val['file_id'].str.replace('Validation PDFs\\', '', regex=False)
pdf_texts_df_val = pdf_texts_df_val.drop(columns=['file_id'])
display(pdf_texts_df_val)

,filename,text,extraction_type
0,C_PROCESO_09-13-197862_113001000_1061376,Ministerio de Hacienda y Crédito Público Repú...,PDF_Reader
1,C_PROCESO_10-13-423871_113002002_2227731,INSTITUTO GEOGRAFICO AGUSTIN CODAZZI TERRITORI...,PDF_Reader
2,C_PROCESO_11-11-539389_115004002_3009336,oo MINISTERIO DE DEFENSA NACIONAL KG V AGENCI...,OCR
3,C_PROCESO_11-12-545661_132003003_2872556,Corporación Autónoma Regional de Cundinamarca ...,PDF_Reader
4,C_PROCESO_11-12-573576_241000001_2988111,fe t be GOBERNACION DEL HUILA CONTRATO INTE...,OCR
...,...,...,...
70,C_PROCESO_22-4-13134789_225290017_106400252,A INSTITUCION EDUCATIVA MUNICIPAL Y Ny LUIS ...,OCR
71,C_PROCESO_22-9-482224_223555011_102972190,Republica de Colombia Departamento de Cordoba...,OCR
72,C_PROCESO_23-12-13445639_215223011_111532588,Cédigo: JUR02P02F05 z . GESTION JURIDICA oe...,OCR
73,C_PROCESO_23-12-13520160_252573011_112659939,REPÚBLICA DE COLOMBIA ALCALDÍA MUNICIPAL PUER...,PDF_Reader


**Now that we have saved the validation texts in another dataframe, we'll substract those observations from the training data.**

In [9]:
pdf_texts_df = pdf_texts_df[~pdf_texts_df['file_id'].str.startswith('Validation PDFs')].reset_index(drop=True)
pdf_texts_df['filename'] = pdf_texts_df['file_id']
pdf_texts_df = pdf_texts_df.drop(columns=['file_id'])
display(pdf_texts_df)

,filename,text,extraction_type
0,C_PROCESO_15-1-142273_254001001_16666533,ADA SANJOSEDE CUCUTA GA Gukd oA BHR hac. CONT...,OCR
1,C_PROCESO_15-1-151124_2110147_17756226,"Elaboró: Lerincon Bogotá D.C., 18 de diciembr...",PDF_Reader
2,C_PROCESO_15-1-151124_2110147_17756718,Asequradora Solidaria Wore mos la Sépti para ...,OCR
3,C_PROCESO_15-1-151124_2110147_17757213,"Elaboró: Lerincon Bogotá D.C., 18 de diciembr...",PDF_Reader
4,C_PROCESO_15-1-151124_2110147_17758709,PositIva COMPANIA bE SEGUROS Bogota D.C. Sef...,OCR
...,...,...,...
93,C_PROCESO_16-4-5588477_225430002_21435367,1OOOCNTFO6 Vi 2 F: 08/2014 ladrid Cus area 86...,OCR
94,C_PROCESO_16-9-424151_266000001_24091356,DEPARTAMENTO DE RISARALDA Secretaria de Educa...,OCR
95,C_PROCESO_17-12-5987688_119004000_27887168,Republica de Colombia Instituto Colombiano d...,OCR
96,C_PROCESO_17-12-6205221_219212019_25765680,RESGUARDO INDIGENA PAEZ DE CORINTO Entidad Púb...,PDF_Reader


**Now we have the extracted text for the training data and validation data. <br>We can retreive more information from the filename.**

In [10]:
for df in [pdf_texts_df, pdf_texts_df_val]: 
    df['tipo_doc'] = df['filename'].str.split('_').str[0]
    df['idproceso'] = df['filename'].str.split('_').str[2]
    df['id_documento'] = df['filename'].str.split('_').str[-2:]
    df['id_documento'] = df['id_documento'].str.join('_')
    df['text']="'" + df['text']
    display(df)

,filename,text,extraction_type,tipo_doc,idproceso,id_documento
0,C_PROCESO_15-1-142273_254001001_16666533,' ADA SANJOSEDE CUCUTA GA Gukd oA BHR hac. CON...,OCR,C,15-1-142273,254001001_16666533
1,C_PROCESO_15-1-151124_2110147_17756226,"' Elaboró: Lerincon Bogotá D.C., 18 de diciemb...",PDF_Reader,C,15-1-151124,2110147_17756226
2,C_PROCESO_15-1-151124_2110147_17756718,' Asequradora Solidaria Wore mos la Sépti para...,OCR,C,15-1-151124,2110147_17756718
3,C_PROCESO_15-1-151124_2110147_17757213,"' Elaboró: Lerincon Bogotá D.C., 18 de diciemb...",PDF_Reader,C,15-1-151124,2110147_17757213
4,C_PROCESO_15-1-151124_2110147_17758709,' PositIva COMPANIA bE SEGUROS Bogota D.C. Se...,OCR,C,15-1-151124,2110147_17758709
...,...,...,...,...,...,...
93,C_PROCESO_16-4-5588477_225430002_21435367,' 1OOOCNTFO6 Vi 2 F: 08/2014 ladrid Cus area 8...,OCR,C,16-4-5588477,225430002_21435367
94,C_PROCESO_16-9-424151_266000001_24091356,' DEPARTAMENTO DE RISARALDA Secretaria de Educ...,OCR,C,16-9-424151,266000001_24091356
95,C_PROCESO_17-12-5987688_119004000_27887168,' Republica de Colombia Instituto Colombiano ...,OCR,C,17-12-5987688,119004000_27887168
96,C_PROCESO_17-12-6205221_219212019_25765680,'RESGUARDO INDIGENA PAEZ DE CORINTO Entidad Pú...,PDF_Reader,C,17-12-6205221,219212019_25765680


,filename,text,extraction_type,tipo_doc,idproceso,id_documento
0,C_PROCESO_09-13-197862_113001000_1061376,' Ministerio de Hacienda y Crédito Público Rep...,PDF_Reader,C,09-13-197862,113001000_1061376
1,C_PROCESO_10-13-423871_113002002_2227731,'INSTITUTO GEOGRAFICO AGUSTIN CODAZZI TERRITOR...,PDF_Reader,C,10-13-423871,113002002_2227731
2,C_PROCESO_11-11-539389_115004002_3009336,' oo MINISTERIO DE DEFENSA NACIONAL KG V AGENC...,OCR,C,11-11-539389,115004002_3009336
3,C_PROCESO_11-12-545661_132003003_2872556,'Corporación Autónoma Regional de Cundinamarca...,PDF_Reader,C,11-12-545661,132003003_2872556
4,C_PROCESO_11-12-573576_241000001_2988111,' fe t be GOBERNACION DEL HUILA CONTRATO INT...,OCR,C,11-12-573576,241000001_2988111
...,...,...,...,...,...,...
70,C_PROCESO_22-4-13134789_225290017_106400252,' A INSTITUCION EDUCATIVA MUNICIPAL Y Ny LUIS...,OCR,C,22-4-13134789,225290017_106400252
71,C_PROCESO_22-9-482224_223555011_102972190,' Republica de Colombia Departamento de Cordob...,OCR,C,22-9-482224,223555011_102972190
72,C_PROCESO_23-12-13445639_215223011_111532588,' Cédigo: JUR02P02F05 z . GESTION JURIDICA o...,OCR,C,23-12-13445639,215223011_111532588
73,C_PROCESO_23-12-13520160_252573011_112659939,' REPÚBLICA DE COLOMBIA ALCALDÍA MUNICIPAL PUE...,PDF_Reader,C,23-12-13520160,252573011_112659939


# Training Data Preparation

**Now that we have the text of each document, which will be used as part of the prompt of each request, to performe the fine-tuning we need its optimal completion output. <br>We import the training output data which has been manually extracted from the documents.**

In [13]:
training_df = pd.merge(pdf_texts_df,pd.read_excel(root + r'\Revision manual_2.xlsx', sheet_name='Revision manual'), on='idproceso', how='inner')
training_df = training_df.fillna('')
for column in training_df:
    if training_df[column].dtype == 'object':  # check if column is of string type
        training_df[column] = training_df[column].str.replace('^ +', '', regex=True)
training_df['CARGO'] = training_df['CARGO'].str.replace('^$', 'No Match', regex=True)
training_df

,filename,text,extraction_type,tipo_doc,idproceso,id_documento,index,contratante,revisor_aprobador_elaborador_1,revisor_aprobador_elaborador_2,revisor_aprobador_elaborador_3,revisor_aprobador_elaborador_4,CARGO,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,C_PROCESO_15-1-142273_254001001_16666533,' ADA SANJOSEDE CUCUTA GA Gukd oA BHR hac. CON...,OCR,C,15-1-142273,254001001_16666533,675175,IRLY YESSENIA SANDOVAL PACHECO,JAIME TENORIO ESTACIO,JORGE VILLAMIZAR JAIMES,,,SECRETARIA PRIVADA ALCALDIA DE CUCUTA,,,,
1,C_PROCESO_15-1-151124_2110147_17756226,"' Elaboró: Lerincon Bogotá D.C., 18 de diciemb...",PDF_Reader,C,15-1-151124,2110147_17756226,883607,DIRECCION NACIONAL DE INTELIGENCIA - DNI,Erika Alba,,,,No Match,,,,
2,C_PROCESO_15-1-151124_2110147_17756718,' Asequradora Solidaria Wore mos la Sépti para...,OCR,C,15-1-151124,2110147_17756718,883607,DIRECCION NACIONAL DE INTELIGENCIA - DNI,Erika Alba,,,,No Match,,,,
3,C_PROCESO_15-1-151124_2110147_17757213,"' Elaboró: Lerincon Bogotá D.C., 18 de diciemb...",PDF_Reader,C,15-1-151124,2110147_17757213,883607,DIRECCION NACIONAL DE INTELIGENCIA - DNI,Erika Alba,,,,No Match,,,,
4,C_PROCESO_15-1-151124_2110147_17758709,' PositIva COMPANIA bE SEGUROS Bogota D.C. Se...,OCR,C,15-1-151124,2110147_17758709,883607,DIRECCION NACIONAL DE INTELIGENCIA - DNI,Erika Alba,,,,No Match,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,C_PROCESO_16-4-5588477_225430002_21435367,' 1OOOCNTFO6 Vi 2 F: 08/2014 ladrid Cus area 8...,OCR,C,16-4-5588477,225430002_21435367,240212,MARIA CRISTINA GARCIA VERGARA,LEIDY ESCOBAR RODELO,HILDA LOPEZ,,,GERENTE (e) E.S.E. HOSPITAL SANTA MATILDE DE M...,,,,
94,C_PROCESO_16-9-424151_266000001_24091356,' DEPARTAMENTO DE RISARALDA Secretaria de Educ...,OCR,C,16-9-424151,266000001_24091356,847467,LILIANA MARIA SANCHEZ VILLADA,LINA MARIA QUINTERO,JULIANA PEREZ MORENO,,,SECRETARIA EDU. RISARALDA,,,,
95,C_PROCESO_17-12-5987688_119004000_27887168,' Republica de Colombia Instituto Colombiano ...,OCR,C,17-12-5987688,119004000_27887168,1037514,JAMES NEY RUIZ GOMEZ,HORACIO ENRIQUE DORADO QUINTERO,CRISTHIAN JAIR NAVIA DAZA,,,DIR. ICBF REGIONAL CAUCA,,,,
96,C_PROCESO_17-12-6205221_219212019_25765680,'RESGUARDO INDIGENA PAEZ DE CORINTO Entidad Pú...,PDF_Reader,C,17-12-6205221,219212019_25765680,567821,HILARIO GUEJIA,,,,,REP. LEGAL RESGUARDO INDIGENA PAEZ DE CORINTO,,,,


**Now that we have the completion data, we generate a single 'completion' column and drop the redundant ones.**

In [14]:
training_df['prompt'] = training_df['text'].str.lower()+'---->'
training_df['completion'] = ' ' + training_df['contratante'].str.title() + '; ' + training_df['CARGO'].str.title() +'\n\n' +training_df['revisor_aprobador_elaborador_1'].str.title() + '\n\n' + training_df['revisor_aprobador_elaborador_2'].str.title() + '\n\n' +training_df['revisor_aprobador_elaborador_3'].str.title() + '\n\n' +training_df['revisor_aprobador_elaborador_4'].str.title() + '\n\n' +training_df['Unnamed: 8'].str.title() + '\n\n' +training_df['Unnamed: 9'].str.title() + '\n\n' +training_df['Unnamed: 10'].str.title() + '\n\n' +training_df['Unnamed: 11'].str.title() + '\n\n' + '\n\n###\n\n'
training_df['completion'] = training_df['completion'].str.replace('(\n\n)+', '\n\n', regex=True)
training_df = training_df.drop(columns=['idproceso', 'text', 'tipo_doc','id_documento','extraction_type','filename','index', 'contratante', 'revisor_aprobador_elaborador_1', 'revisor_aprobador_elaborador_2','revisor_aprobador_elaborador_3','revisor_aprobador_elaborador_4', 'CARGO', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'])
training_df = training_df.drop_duplicates().reset_index(drop=True)
training_df

,prompt,completion
0,' ada sanjosede cucuta ga gukd oa bhr hac. con...,Irly Yessenia Sandoval Pacheco ; Secretaria P...
1,"' elaboró: lerincon bogotá d.c., 18 de diciemb...",Direccion Nacional De Inteligencia - Dni ; No...
2,' asequradora solidaria wore mos la sépti para...,Direccion Nacional De Inteligencia - Dni ; No...
3,' positiva compania be seguros bogota d.c. se...,Direccion Nacional De Inteligencia - Dni ; No...
4,' :47ta4 institution universitaria contrato de...,Maria Elena Garcia Agudelo ; Jefe De Of. Ases...
...,...,...
91,' 1ooocntfo6 vi 2 f: 08/2014 ladrid cus area 8...,Maria Cristina Garcia Vergara ; Gerente (E) E...
92,' departamento de risaralda secretaria de educ...,Liliana Maria Sanchez Villada ; Secretaria Ed...
93,' republica de colombia instituto colombiano ...,James Ney Ruiz Gomez ; Dir. Icbf Regional Cau...
94,'resguardo indigena paez de corinto entidad pú...,Hilario Guejia ; Rep. Legal Resguardo Indigen...


**Because the full texts can be very long and make the requests too expensive, we define a function to split the texts into smaller chunks.**

In [15]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.9 * n and 1.1 * n tokens
        j = min(i + int(1.1 * n), len(tokens))
        while j > i + int(0.9 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.9 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

**Next we limit the prompt to only contain the first n tokens and last n tokens of the texts.**

In [16]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('curie')

prompt=[]

for i in tqdm(range(len(training_df['prompt']))):
    chunks = create_chunks(training_df.loc[i,'prompt'], 800, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    if len(text_chunks)>=2:
        prompt.append(' '.join(text_chunks[:1]+text_chunks[-1:]))
    else:
        prompt.append(' '.join(text_chunks))
training_df['prompt'] = prompt

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [00:02<00:00, 36.92it/s]


In [17]:
training_df.to_json(root + r'\training_final.jsonl', orient='records', lines=True)

In [18]:
training_df

,prompt,completion
0,' ada sanjosede cucuta ga gukd oa bhr hac. con...,Irly Yessenia Sandoval Pacheco ; Secretaria P...
1,"' elaboró: lerincon bogotá d.c., 18 de diciemb...",Direccion Nacional De Inteligencia - Dni ; No...
2,' asequradora solidaria wore mos la sépti para...,Direccion Nacional De Inteligencia - Dni ; No...
3,' positiva compania be seguros bogota d.c. se...,Direccion Nacional De Inteligencia - Dni ; No...
4,' :47ta4 institution universitaria contrato de...,Maria Elena Garcia Agudelo ; Jefe De Of. Ases...
...,...,...
91,' 1ooocntfo6 vi 2 f: 08/2014 ladrid cus area 8...,Maria Cristina Garcia Vergara ; Gerente (E) E...
92,' departamento de risaralda secretaria de educ...,Liliana Maria Sanchez Villada ; Secretaria Ed...
93,' republica de colombia instituto colombiano ...,James Ney Ruiz Gomez ; Dir. Icbf Regional Cau...
94,'resguardo indigena paez de corinto entidad pú...,Hilario Guejia ; Rep. Legal Resguardo Indigen...


### OpenAI's Data Preparation Tool

In [19]:
!pip install --upgrade openai

In [20]:
!openai tools fine_tunes.prepare_data -f training_final.jsonl -q

Analyzing...

- Your file contains 96 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix ` ---->`
- All prompts start with prefix `'`
- All completions end with suffix `\n\n###\n\n`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "training_final.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` ---->` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n\n###\n\n"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 3.76 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


# Fine Tuning OpenAI's Curie Model

**In order to fine tune the model we first need to create an OpenAI API Key in our account and set it in the environment as shown bellow.**

In [21]:
os.environ['OPENAI_API_KEY'] = ""

In [22]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [23]:
!openai api fine_tunes.create -t "training_final.jsonl" -m "curie" --suffix "curie_final" 

Uploaded file from training_final.jsonl: file-nnciT7FxVa1jEE2HcPuFV1y6
Created fine-tune: ft-beDYtOF7xyNQtwbfA5LseQXt
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-06-02 02:35:23] Created fine-tune: ft-beDYtOF7xyNQtwbfA5LseQXt

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-beDYtOF7xyNQtwbfA5LseQXt




Upload progress:   0%|          | 0.00/336k [00:00<?, ?it/s]
Upload progress: 100%|##########| 336k/336k [00:00<00:00, 336Mit/s]


In [ ]:
raise SystemExit("Don't forget to change the finetune id so that it matches with the one we just created.")

In [25]:
!openai api fine_tunes.follow -i ft-beDYtOF7xyNQtwbfA5LseQXt

[2023-06-02 02:35:23] Created fine-tune: ft-beDYtOF7xyNQtwbfA5LseQXt
[2023-06-02 02:37:46] Fine-tune costs $1.29
[2023-06-02 02:37:46] Fine-tune enqueued. Queue number: 1
[2023-06-02 02:37:48] Fine-tune is in the queue. Queue number: 0
[2023-06-02 02:37:54] Fine-tune started
[2023-06-02 02:39:12] Completed epoch 1/4
[2023-06-02 02:39:30] Completed epoch 2/4
[2023-06-02 02:39:48] Completed epoch 3/4
[2023-06-02 02:40:06] Completed epoch 4/4
[2023-06-02 02:40:27] Uploaded model: curie:ft-personal:curie-final-2023-06-02-07-40-26
[2023-06-02 02:40:28] Uploaded result file: file-PwMKFYnn6QjhaSJPJmO7Hcc2
[2023-06-02 02:40:28] Fine-tune succeeded


Traceback (most recent call last):
  File "C:\Users\matia\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\matia\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\matia\anaconda3\Scripts\openai.exe\__main__.py", line 7, in <module>
  File "C:\Users\matia\anaconda3\lib\site-packages\openai\_openai_scripts.py", line 78, in main
    args.func(args)
  File "C:\Users\matia\anaconda3\lib\site-packages\openai\cli.py", line 523, in follow
    cls._stream_events(args.id)
  File "C:\Users\matia\anaconda3\lib\site-packages\openai\cli.py", line 566, in _stream_events
    sys.stdout.write("\nJob complete! Status: succeeded \U0001f389")
  File "C:\Users\matia\anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f389' in position 34: character maps to 

In [26]:
!openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1684798825,
      "fine_tuned_model": "babbage:ft-personal:babbage-prueba-2023-05-22-23-44-50",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-x97ZKe1EQ23E3QenXg5VWNY0",
      "model": "babbage",
      "object": "fine-tune",
      "organization_id": "org-Pp6StbgvoyY42pkMUAcanFUD",
      "result_files": [
        {
          "bytes": 11971,
          "created_at": 1684799091,
          "filename": "compiled_results.csv",
          "id": "file-UAAa3mxuFRDG2ReRe41J0UIR",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 460259,
          "created_at": 1684798825,
          "filename": "training_data_prepared.jsonl",
          "id": "file-J1e

# Validation Data Preparation

In [27]:
validation_df = pdf_texts_df_val

In [28]:
validation_df['prompt'] = validation_df['text'].str.lower()
validation_df = validation_df.drop(columns=[ 'text', 'tipo_doc','id_documento','extraction_type','filename'])
validation_df = validation_df.drop_duplicates().reset_index(drop=True)
validation_df

,idproceso,prompt
0,09-13-197862,' ministerio de hacienda y crédito público rep...
1,10-13-423871,'instituto geografico agustin codazzi territor...
2,11-11-539389,' oo ministerio de defensa nacional kg v agenc...
3,11-12-545661,'corporación autónoma regional de cundinamarca...
4,11-12-573576,' fe t be gobernacion del huila contrato int...
...,...,...
70,22-4-13134789,' a institucion educativa municipal y ny luis...
71,22-9-482224,' republica de colombia departamento de cordob...
72,23-12-13445639,' cédigo: jur02p02f05 z . gestion juridica o...
73,23-12-13520160,' república de colombia alcaldía municipal pue...


In [29]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('curie')

prompt=[]

for i in tqdm(range(len(validation_df['prompt']))):
    chunks = create_chunks(validation_df.loc[i,'prompt'], 600, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    if len(text_chunks)>=2:
        prompt.append(' '.join(text_chunks[:1]+text_chunks[-1:]))
    else:
        prompt.append(' '.join(text_chunks))
validation_df['prompt'] = prompt

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:03<00:00, 21.59it/s]


# Running Fine Tuned Model

**Now that we have fine tuned our model, we can test it by running it on the validation data.**

## Prompt and Model Design

**We specify the prompt to be used in each request. <br>In this case, we just want to input the limited text of the document.**

In [30]:
# Example prompt - 
document = '<document>'
template_prompt=f'''{document}---->'''
print(template_prompt)

<document>---->


**We also define the function that inserts the extracted text into the prompt design and runs the OpenAI completion model.**

In [ ]:
raise SystemExit("Don't forget to change the model name so that it matches with the one we just created.")

In [31]:
def extract_chunk(document,template_prompt):
    
    prompt=template_prompt.replace('<document>',document)
    
    try:
        response = openai.Completion.create(
        model='curie:ft-personal:curie-final-2023-06-02-07-40-26', 
        prompt=prompt,
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n\n###\n\n"]
        )
        return response['choices'][0]['text']
    
    except openai.error.RateLimitError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"Rate limit error. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return extract_chunk(document,template_prompt)

    except openai.error.APIError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"API error occurred. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return extract_chunk(document,template_prompt)

    except OSError as e:
        retry_time = 1  # Adjust the retry time as needed
        print(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        return extract_chunk(document,template_prompt)

In [32]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('curie')

results = []

for i in tqdm(range(len(validation_df))):

    #for chunk in text_chunks:
    results.append(extract_chunk(validation_df.loc[i,'prompt'],template_prompt))
    #print(chunk)

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:48<00:00,  1.56it/s]


In [33]:
validation_df['gpt_answer'] = results
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 2000):  # more options can be specified also
    display(validation_df)

,idproceso,prompt,gpt_answer
0,09-13-197862,"' ministerio de hacienda y crédito público república de colombia 10309pjsaecom8605253671i05acmg/anmg/vapv/gmr página 1 de 12 ministerio de hacienda y crédito público contratante: contratista: nit no.: objeto: plazo: valor: la nación ministerio de hacienda y crédito público servicios asociados especializados en comunicaciones ltda 860.525.3671 prestar el servicio de mensajería especializada para la entrega de comunicaciones oficiales del ministerio de hacienda y crédito público. hasta el 31 de julio de 2010 33000.000 incluido el iva, equivalentes a 66.41 s.m.m.l.v. bogotá d.c., señor: german echeverry ocampo servicios asociados especializados en comunicaciones ltda. gerente ciudad. por medio de la presente orden, de código , me permito comunicarle que la nación ministerio de hacienda y crédito público, que para los efectos del presente documento se denominará el ministerio, por intermedio de este despacho, autoriza la prestación de servicios que a continuación se detalla, y se regirá por los siguientes términos, previas las siguientes consideraciones: 1. que de acuerdo a los estudios previos elaborados por la subdirección de servicios del ministerio, esta orden de prestación de servicios tiene como justificación lo siguiente: el artículo 37 de la ley 80 de 1993, establece: los servicios postales comprenden la prestación de los servicios de correo y del servicio de mensajería especializada. se entiende por servicio de correo la prestación de los servicios de giros postales y telegráficos, así como el recibo, clasificación y entrega de envíos de correspondencia y otros objetos postales, transportados vía superficie y aérea, dentro del territorio nacional. el servicio de correo internacional se prestará de acuerdo con los convenios y acuerdos internacionales suscritos con la unión postal universal y los países miembros. domicilio contractual: para efectos de la ejecución de la presente orden, el domicilio contractual será la ciudad de bogotá d.c. 16. lugar de ejec...",German Echeverry Ocampo ; Gerente Ciudad\n\nGloria Elvira Ortiz
1,10-13-423871,"'instituto geografico agustin codazzi territorial meta sistema gestión de calidad certificado villavicencio carrera 33a no. 3714 teléfonos 6622913 fax: 6624240 web: www.igac.gov.co pagina 1 de 5 valor : sistemas y distribuciones formacon ltda nit: 830.006.8004 dirección: calle 14 1250 of. 205 b bogotá d.c. teléfono: 091 2830292 3375244 2838895 3415857 email: formaconltdaetb.net.co 17,627,444.00 contrato de compraventa 646 fecha: 15 de diciembre de 2010 entre los suscritos a saber: osmel santiago rojas benito, mayor de edad e identificado con la cédula de ciudadanía número 19.220.191 de bogotá, quien obra como director territorial meta del instituto geográfico agustín codazzi, nit: 899.999.004 9, establecimiento público regido por el decreto 2113 de 1992, en uso de las facultades conferidas por la resolución 196 del 02 de abril del 2009, emanada de la dirección general del instituto geográfico agustín codazzi y con sujeción a la ley 80 de 1993 y quien en adelante se denominará el instituto y danilo carvajal panqueva, mayor de edad e identificado con la cédula de ciudadanía número 72.242.464 de bogotá d.c, quien obra como gerente y representante legal de sistemas y distribuciones formacon ltda, nit: 830.006.8004, inscrita en el registro de proponentes bajo el no. 13026, inscrita en el rut, quien manifiesta bajo juramento que ni la sociedad que representa ni él se hallan incursos en ninguna de las inhabilidades e incompatibilidades relacionadas en el artículo 8 de la ley 80 de 1993 y que no aparece en el boletín de responsables fiscales, quien en este documento se denominará el contratista, en desarrollo del proceso de contratación de selección abreviada de mínima cuantía no. 24 de 2010, se acuerda celebrar el presente contrato que consta en las siguientes cláusulas: primera. gov.co continuación contrato no.celebrado entre instituto geografico agu

In [35]:
print(validation_df['gpt_answer'][1])

 Osmel Santiago Rojas Benito ; Dir. Territorial Meta Igac

Clara Ruth Garnica 

Diana Umaña 
